In [8]:
import pandas as pd

In [9]:
df = pd.read_json("converted_hello_faqs.json", orient="records")
df.head()

,faq_id,converted_question,converted_answer,original_question,original_answer,include_ask_contact_info,user_gender,user_refers_to_assistant,user_refers_to_themselves,assistant_refers_to_user,assistant_refers_to_themselves,error
0,-1,Anh chào em,Chào anh! Anh có cần hỗ trợ gì không ạ? Hiện b...,Xin chào,Chào anh/chị! Anh/chị có cần hỗ trợ gì không ạ...,False,male,em,anh,anh,em,NaN
1,-1,Xin chào,Chào anh! Anh có cần hỗ trợ gì không ạ? Hiện b...,Xin chào,Chào anh/chị! Anh/chị có cần hỗ trợ gì không ạ...,False,male,not provide,not provide,anh,em,NaN
2,-1,Em xin chào anh,Chào anh! Anh có cần hỗ trợ gì không ạ? Hiện b...,Xin chào,Chào anh/chị! Anh/chị có cần hỗ trợ gì không ạ...,False,male,anh,em,anh,em,NaN
3,-1,Em xin chào chi,Chào anh! Anh có cần hỗ trợ gì không ạ? Hiện b...,Xin chào,Chào anh/chị! Anh/chị có cần hỗ trợ gì không ạ...,False,male,chị,em,anh,em,NaN
4,-1,Chú xin chào cháu,Chào chú! Chú có cần hỗ trợ gì không ạ? Hiện b...,Xin chào,Chào anh/chị! Anh/chị có cần hỗ trợ gì không ạ...,False,male,cháu,chú,chú,cháu,NaN


In [32]:
import jsonlines

def write_to_jsonlines(
    data: dict | list[dict], file_path: str = "fine_tune_tone_dataset.jsonl"
):
    with jsonlines.open(file_path, "a") as writer:
        if type(data) is dict:
            writer.write(data)
        else:
            writer.write_all(data)

def to_openai_messages(question: str, answer: str, gender: str) -> list[dict]:
    return [
        {
            "role": "system",
            "content": """# ROLE
You are professional sales consultant staff for a online store (that provides technology products) in Vietnam.""",
        },
        {
            "role": "system",
            "content": f"""## BASE KNOWLEDGE:
- User gender: {gender}
- Information about your store:
   - Name: FPTShop
   - Location: https://fptshop.com.vn/cua-hang
   - Hotline: 1800.6601
   - Website: [FPTShop](https://fptshop.com.vn)
   - Customer service email: cskh@fptshop.com""",
        },
        {
            "role": "user",
            "content": question,
        },
        {
            "role": "system",
            "content": f"This is the latest user's message: {question}",
        },
        {
            "role": "system",
            "content": """## RULES:
- Don't talk nonsense and make up facts.
- Use only the Vietnamese language in your response. Always refer to yourself using 'em' pronoun, except when addressing the user as 'cô,' 'chú,' or 'bác,' in which case use 'con' to refer to yourself. Address the user based on how they refer to themselves: if they explicitly use 'cô' or 'chú,' address them with that term. If their preferred address term cannot be determined from their self-reference, use 'anh' for males, 'chị' for females based on their provided gender, or the polite neutral term 'anh/chị' if gender is unknown or not provided. Do not use terms like 'anh' or 'chị' for yourself, as this would imply a higher status.""",
        },
        {
            "role": "system",
            "content": """## WORKFLOW:
**Step 1:** Read carefully the latest user message and the context provided by the user.
**Step 2:** Suggest some types of products that the user might be interested in.
example: 'Anh/chị có cần hỗ trợ gì không ạ? Hiện bên em có một số loại sản phẩm như: điện thoại, laptop,...'""",
        },
        {
            "role": "assistant",
            "content": answer,
        }
    ]


def to_openai_messages_for_DPO(question: str, expect_answer: str, gender: str, current_answer: str) -> dict:
    return {
  "input": {"messages" : [
        {
            "role": "system",
            "content": """# ROLE
You are professional sales consultant staff for a online store (that provides technology products) in Vietnam.""",
        },
        {
            "role": "system",
            "content": f"""## BASE KNOWLEDGE:
- User gender: {gender}
- Information about your store:
   - Name: FPTShop
   - Location: https://fptshop.com.vn/cua-hang
   - Hotline: 1800.6601
   - Website: [FPTShop](https://fptshop.com.vn)
   - Customer service email: cskh@fptshop.com""",
        },
        {
            "role": "user",
            "content": question,
        },
        {
            "role": "system",
            "content": f"This is the latest user's message: {question}",
        },
        {
            "role": "system",
            "content": """## RULES:
- Don't talk nonsense and make up facts.
- Use only the Vietnamese language in your response. Always refer to yourself using 'em' pronoun, except when addressing the user as 'cô,' 'chú,' or 'bác,' in which case use 'con' to refer to yourself. Address the user based on how they refer to themselves: if they explicitly use 'cô' or 'chú,' address them with that term. If their preferred address term cannot be determined from their self-reference, use 'anh' for males, 'chị' for females based on their provided gender, or the polite neutral term 'anh/chị' if gender is unknown or not provided. Do not use terms like 'anh' or 'chị' for yourself, as this would imply a higher status.""",
        },
        {
            "role": "system",
            "content": """## WORKFLOW:
**Step 1:** Read carefully the latest user message and the context provided by the user.
**Step 2:** Suggest some types of products that the user might be interested in.
example: 'Anh/chị có cần hỗ trợ gì không ạ? Hiện bên em có một số loại sản phẩm như: điện thoại, laptop,...'""",
        },
    ]},
            "preferred_output": [
    {
      "role": "assistant",
      "content": expect_answer
    }
  ],
  "non_preferred_output": [
    {
      "role": "assistant",
      "content": current_answer
    }
  ]
    }


failed_hello_df = pd.read_json("failed_addressing_analysis.json", orient="records")

for row in list(failed_hello_df.itertuples(index=False)):
    converted_question = getattr(row, "converted_question", "")
    converted_answer = getattr(row, "converted_answer", "")
    include_contact_info = getattr(row, "include_ask_contact_info", False)
    current_answer = getattr(row, "actual_answers", [])[0]
    write_to_jsonlines(to_openai_messages_for_DPO(
                question=converted_question,
                expect_answer=converted_answer,
                gender=getattr(row, "user_gender", "unknown"),
                current_answer=current_answer,
            )
        ,
        file_path="fine_tune_tone_hello_DPO_dataset.jsonl",
    )

# for row in list(df.itertuples(index=False))[:710]:
#     converted_question = getattr(row, "converted_question", "")
#     converted_answer = getattr(row, "converted_answer", "")
#     original_question = getattr(row, "original_question", "")
#     original_answer = getattr(row, "original_answer", "")

#     include_contact_info = getattr(row, "include_ask_contact_info", False)
#     write_to_jsonlines(
#         {
#             "messages": to_openai_messages(
#                 question=converted_question,
#                 answer=converted_answer,
#                 gender=getattr(row, "user_gender", "unknown"),
#             )
#         },
#         file_path="fine_tune_tone_hello_dataset.jsonl",
# )

In [11]:
from service.openai import _client
import json


def detect_question_patterns(question: str) -> dict:
    """
    Detect addressing patterns in the customer's question.
    
    Args:
        question: Customer's question
    
    Returns:
        Dictionary with addressing pattern analysis for the question
    """
    prompt = f"""
Bạn là một chuyên gia phân tích ngôn ngữ tiếng Việt chuyên sâu về cách xưng hô. Hãy phân tích chi tiết cách xưng hô trong câu hỏi của khách hàng sau:

CÂU HỎI CỦA KHÁCH HÀNG:
"{question}"

NHIỆM VỤ PHÂN TÍCH:

1. PHÂN TÍCH CÁCH XƯNG HÔ CỦA KHÁCH HÀNG TRONG CÂU HỎI:
   - Cách khách hàng gọi BẢN THÂN mình trong câu hỏi (ví dụ: "tôi", "mình", "em", "anh", "chị", "cô", "chú", "bác")
   - Cách khách hàng gọi NHÂN VIÊN TƯ VẤN trong câu hỏi (ví dụ: "anh", "chị", "em", "cô", "chú", "bác", "shop", "bạn")

ĐỊNH DẠNG TRẢ VỀ JSON:
{{
    "user_refers_to_assistant": "cách khách hàng gọi nhân viên tư vấn",
    "user_refers_to_themselves": "cách khách hàng gọi bản thân"
}}

Chỉ trả về JSON, không có text giải thích thêm.
"""

    try:
        response = _client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {
                    "role": "system",
                    "content": "Bạn là chuyên gia phân tích ngôn ngữ tiếng Việt về cách xưng hô. Phân tích chính xác và trả về JSON theo đúng format yêu cầu.",
                },
                {"role": "user", "content": prompt},
            ],
            response_format={
                "type": "json_schema",
                "json_schema": {
                    "name": "question_addressing_analysis",
                    "description": "Analysis of addressing patterns in the customer's question",
                    "strict": True,
                    "schema": {
                        "type": "object",
                        "properties": {
                            "user_refers_to_assistant": {
                                "type": "string",
                                "description": "How the customer addresses the sales consultant",
                            },
                            "user_refers_to_themselves": {
                                "type": "string",
                                "description": "How the customer refers to themselves",
                            },
                        },
                        "additionalProperties": False,
                        "required": [
                            "user_refers_to_assistant",
                            "user_refers_to_themselves",
                        ],
                    },
                },
            },
            temperature=0,
        )

        return json.loads(response.choices[0].message.content or "{}")

    except Exception as e:
        print(f"Error detecting question patterns: {e}")
        raise e


def detect_answer_patterns(answer: str) -> dict:
    """
    Detect addressing patterns in the sales consultant's answer.
    
    Args:
        answer: Sales consultant's answer
    
    Returns:
        Dictionary with addressing pattern analysis for the answer
    """
    prompt = f"""
Bạn là một chuyên gia phân tích ngôn ngữ tiếng Việt chuyên sâu về cách xưng hô. Hãy phân tích chi tiết cách xưng hô trong câu trả lời của nhân viên tư vấn sau:

CÂU TRẢ LỜI CỦA NHÂN VIÊN TƯ VẤN:
"{answer}"

NHIỆM VỤ PHÂN TÍCH:

1. PHÂN TÍCH CÁCH XƯNG HÔ CỦA NHÂN VIÊN TƯ VẤN TRONG CÂU TRẢ LỜI:
   - Cách nhân viên tư vấn gọi BẢN THÂN mình trong câu trả lời (ví dụ: "tôi", "mình", "em", "con", "cháu")
   - Cách nhân viên tư vấn gọi KHÁCH HÀNG trong câu trả lời (ví dụ: "anh/chị", "em", "cô/chú", "bác", "quý khách")

ĐỊNH DẠNG TRẢ VỀ JSON:
{{
    "assistant_refers_to_user": "cách nhân viên tư vấn gọi khách hàng",
    "assistant_refers_to_themselves": "cách nhân viên tư vấn gọi bản thân"
}}

Chỉ trả về JSON, không có text giải thích thêm.
"""

    try:
        response = _client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {
                    "role": "system",
                    "content": "Bạn là chuyên gia phân tích ngôn ngữ tiếng Việt về cách xưng hô. Phân tích chính xác và trả về JSON theo đúng format yêu cầu.",
                },
                {"role": "user", "content": prompt},
            ],
            response_format={
                "type": "json_schema",
                "json_schema": {
                    "name": "answer_addressing_analysis",
                    "description": "Analysis of addressing patterns in the sales consultant's answer",
                    "strict": True,
                    "schema": {
                        "type": "object",
                        "properties": {
                            "assistant_refers_to_user": {
                                "type": "string",
                                "description": "How the sales consultant addresses the customer",
                            },
                            "assistant_refers_to_themselves": {
                                "type": "string",
                                "description": "How the sales consultant refers to themselves",
                            },
                        },
                        "additionalProperties": False,
                        "required": [
                            "assistant_refers_to_user",
                            "assistant_refers_to_themselves",
                        ],
                    },
                },
            },
            temperature=0,
        )

        return json.loads(response.choices[0].message.content or "{}")

    except Exception as e:
        print(f"Error detecting answer patterns: {e}")
        raise e


def analyze_addressing_patterns(question: str, answer: str) -> dict:
    """
    Analyze Vietnamese addressing patterns between customer and sales consultant.
    
    Args:
        question: Customer's question
        answer: Sales consultant's answer
    
    Returns:
        Dictionary with addressing pattern analysis
    """
    question_patterns = detect_question_patterns(question)
    answer_patterns = detect_answer_patterns(answer)

    return {**question_patterns, **answer_patterns}




In [12]:
# Only one implementation of addressing_formal_metric should exist in the notebook.
def addressing_formal_metric(
    actual_output: str, expected_assistant_refers_to_user: str, expected_assistant_refers_to_themselves: str,
    reason: bool = True
) -> float:
    """
    Calculate the addressing formal metric based on the assistant's response.
    
    Args:
        actual_output: The actual output from the assistant
        expected_assistant_refers_to_user: Expected way the assistant refers to the user
        expected_assistant_refers_to_themselves: Expected way the assistant refers to themselves
    Returns:
        float: Formality score (1.0 for perfect match, 0.0 for no match)
    """
    try:
        addressing_in_actual_output = detect_answer_patterns(actual_output)
        actual_assistant_refers_to_user = addressing_in_actual_output.get(
            "assistant_refers_to_user", "not provided"
        )
        actual_assistant_refers_to_themselves = addressing_in_actual_output.get(
            "assistant_refers_to_themselves", "not provided"
        )
        
        assistant_refers_to_user_match = False
        assistant_refers_to_themselves_match = False
        
        if not actual_assistant_refers_to_user or actual_assistant_refers_to_user == "not provided" or expected_assistant_refers_to_user.lower() in actual_assistant_refers_to_user.lower() or \
             actual_assistant_refers_to_user.lower() in expected_assistant_refers_to_user.lower():
            assistant_refers_to_user_match = True
        elif reason:
            print(f"Expected user reference: {expected_assistant_refers_to_user}, Actual: {actual_assistant_refers_to_user}")
            print(f"Expected self-reference: {expected_assistant_refers_to_themselves}, Actual: {actual_assistant_refers_to_themselves}")
        
        if not actual_assistant_refers_to_themselves or actual_assistant_refers_to_themselves == "not provided" or expected_assistant_refers_to_themselves.lower() in actual_assistant_refers_to_themselves.lower() or \
             actual_assistant_refers_to_themselves.lower() in expected_assistant_refers_to_themselves.lower():
            assistant_refers_to_themselves_match = True
        elif reason:
            print(f"Expected user reference: {expected_assistant_refers_to_user}, Actual: {actual_assistant_refers_to_user}")
            print(f"Expected self-reference: {expected_assistant_refers_to_themselves}, Actual: {actual_assistant_refers_to_themselves}")
        
        if assistant_refers_to_user_match and assistant_refers_to_themselves_match:
            return 1.0
        elif not assistant_refers_to_user_match and not assistant_refers_to_themselves_match:
            return 0.0
        else:
            return 0.5

    except Exception as e:
        print(f"Error calculating addressing formal metric: {e}")
        return 0.0

In [18]:
from service.store_chatbot_v2 import gen_answer, ConfigModel
from repositories.user import create as create_user, CreateUserModel, UserRole
from repositories.thread import create as create_thread, CreateThreadModel
from uuid import uuid4

def evaluate_addressing_multi_turns(
    questions: list[str],
    expected_assistant_refers: list[dict[str, str]],
    gender: str = "unknown",
    model: str = "gpt-4o-mini",
):
    user = create_user(
        CreateUserModel(user_name=str(uuid4()), role=UserRole.chainlit_user, gender=gender)
    )
    thread = create_thread(CreateThreadModel(user_id=user.id, name=user.user_name))
    actual_answers = []
    conversation_history = []
    for question in questions:
        conversation_history.append(
            {
                "role": "user",
                "content": question,
            }
        )
        answer = gen_answer(
            history=conversation_history,
            user_id=user.id,
            thread_id=thread.id,
            config=ConfigModel(
                response=model,
                use_fine_tune_tone=True,
            ),
        )
        conversation_history.append(
            {
                "role": "assistant",
                "content": answer,
            }
        )
        actual_answers.append(answer)
    scores = []
    for i, actual_answer in enumerate(actual_answers):
        expected_assistant_refers_to_user = expected_assistant_refers[i].get(
            "assistant_refers_to_user", "not provided"
        )
        expected_assistant_refers_to_themselves = expected_assistant_refers[i].get(
            "assistant_refers_to_themselves", "not provided"
        )
        score = addressing_formal_metric(
            actual_output=actual_answer,
            expected_assistant_refers_to_user=expected_assistant_refers_to_user,
            expected_assistant_refers_to_themselves=expected_assistant_refers_to_themselves,
        )
        scores.append(score)
        
    return {
        "scores": scores,
        "average_score": sum(scores) / len(scores) if scores else 0.0,
        "actual_answers": actual_answers,
        "user_id": user.id,
        "thread_id": thread.id,
    }
    


In [29]:
scores = []
failed_df = None

for row in list(df.itertuples(index=False)):
    converted_question = getattr(row, "converted_question", "")
    converted_answer = getattr(row, "converted_answer", "")
    expected_assistant_refers_to_user = getattr(
        row, "assistant_refers_to_user", "not provided"
    )
    expected_assistant_refers_to_themselves = getattr(
        row, "assistant_refers_to_themselves", "not provided"
    )
    expected_assistant_refers = [
        {
            "assistant_refers_to_user": expected_assistant_refers_to_user,
            "assistant_refers_to_themselves": expected_assistant_refers_to_themselves,
        }
    ]
    gender = getattr(row, "user_gender", "unknown")

    measure_result = evaluate_addressing_multi_turns(
        questions=[converted_question],
        expected_assistant_refers=expected_assistant_refers,
        gender=gender,
        model="ft:gpt-4o-mini-2024-07-18:personal:faq-3:BijizY61",
    )

    score = measure_result["average_score"]

    if score < 1:
        if failed_df is None:
            failed_df = pd.DataFrame(columns=list(df.columns) + ["score", "actual_answers"])
        row_dict = dict(row._asdict())
        row_dict["score"] = score
        row_dict["actual_answers"] = measure_result["actual_answers"]
        failed_df = pd.concat(
            [failed_df, pd.DataFrame([row_dict])],
            ignore_index=True
        )

    scores.append(score)
    print(f"Score for question '{converted_question}': {score}")


average_score = sum(scores) / len(scores) if scores else 0.0
print(f"Average score for all questions: {average_score}")

2025-06-15 15:57:48 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
User request: {'user_demand': <ProductType.UNDETERMINED: 'undetermined'>, 'user_info': {'phone_number': None, 'email': None}}
Detect demand response: type='finished' content='The user request has been successfully processed.' instructions=[] UserIntent(is_user_needs_other_suggestions=False, product_type=None)
2025-06-15 15:57:49 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-06-15 15:57:51 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Updating user memory with id: 215ebdc1-ff12-456e-b8e8-27fcc36a2e32 and data: {'user_demand': None, 'product_name': None, 'brand_code': None, 'brand_name': None, 'min_price': None, 'max_price': None, 'phone_number': None, 'email': None, 'intent': {'is_user_needs_other_suggestions': False, 'product_type': None}, 'current_filter': {'product_name': None}, 'consultation_status': {'is_recomme

/tmp/ipykernel_23099/4044890792.py:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  failed_df = pd.concat(


2025-06-15 15:58:10 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
User request: {'user_demand': <ProductType.UNDETERMINED: 'undetermined'>, 'user_info': {'phone_number': None, 'email': None}}
Detect demand response: type='finished' content='The user request has been successfully processed.' instructions=[] UserIntent(is_user_needs_other_suggestions=False, product_type=None)
2025-06-15 15:58:11 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-06-15 15:58:12 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Updating user memory with id: 30942c0b-8810-49a3-a0c3-de080a52321a and data: {'user_demand': None, 'product_name': None, 'brand_code': None, 'brand_name': None, 'min_price': None, 'max_price': None, 'phone_number': None, 'email': None, 'intent': {'is_user_needs_other_suggestions': False, 'product_type': None}, 'current_filter': {'product_name': None}, 'consultation_status': {'is_recomme

In [30]:
failed_df.to_json(
    "failed_addressing_analysis_faq_3.json",
    orient="records",
    force_ascii=False,
    indent=4,
)


In [24]:
faqs_training_df = pd.read_json("converted_faqs_710.json", orient="records")
faq_ids = faqs_training_df['faq_id'].unique().tolist()
faq_ids

[3, 4, 5, 6, 7]

In [ ]:
# convert faqs_training_df into 5 df by faq_id
for faq_id in faq_ids:
    faq_df = faqs_training_df[faqs_training_df['faq_id'] == faq_id]
    faq_df = faq_df.drop(columns=['faq_id'])
    faq_df.to_json(
        f"fine_tune_tone_faq_{faq_id}.json",
        orient="records",
        force_ascii=False,
        indent=2,
    )

In [26]:
from datetime import datetime
from pydantic import BaseModel
from openai.types.chat import ChatCompletionMessageParam
from service.faq import search as search_faq

class Instruction(BaseModel):
    content: str
    examples: list[str] = []


class SystemPromptConfig(BaseModel):
    role: str = "You are professional sales consultant staff for a store."

    base_knowledge: list[str] = [
        (
            "Information about your phone store:\n"
            "   - Name: FPTShop\n"
            "   - Location: https://fptshop.com.vn/cua-hang\n"
            "   - Hotline: 1800.6601\n"
            "   - Website: [FPTShop](https://fptshop.com.vn)\n"
            "   - Customer service email: cskh@fptshop.com\n"
        ),
    ]
    rules: list[str] = [
        "Don't talk nonsense and make up facts.",
        "Use only the Vietnamese language in your response. Always refer to yourself using 'em' pronoun, except when addressing the user as 'cô,' 'chú,' or 'bác,' in which case use 'con' to refer to yourself. Address the user based on how they refer to themselves: if they explicitly use 'cô' or 'chú,' address them with that term. If their preferred address term cannot be determined from their self-reference, use 'anh' for males, 'chị' for females based on their provided gender, or the polite neutral term 'anh/chị' if gender is unknown or not provided. Do not use terms like 'anh' or 'chị' for yourself, as this would imply a higher status.",
    ]

    instructions: list[Instruction] = []

    def base_knowledge_to_string(self) -> str:
        return "\n".join([f"- {knowledge}" for knowledge in self.base_knowledge])

    def rules_to_string(self) -> str:
        return "\n".join([f"- {rule}" for rule in self.rules])

    def instructions_to_string(self) -> str:
        if len(self.instructions) == 0:
            return ""

        text = ""
        for i, instruction in enumerate(self.instructions):
            text += f"{i}. {instruction.content}:\n"
            if not instruction.examples:
                continue
            if len(instruction.examples) == 1:
                text += f"Example: {instruction.examples[0]}\n"
            else:
                text += "Examples:\n"
                text += "\n".join(
                    [f"   - {example}" for example in instruction.examples]
                )

        return text

    def get_openai_messages(
        self,
        conversation_messages: list[ChatCompletionMessageParam],
    ) -> list[ChatCompletionMessageParam]:

        role_task_skill_message: ChatCompletionMessageParam = {
            "role": "system",
            "content": f"""# ROLE
{self.role}
""",
        }
        base_knowledge_message: ChatCompletionMessageParam = {
            "role": "system",
            "content": f"""## BASE KNOWLEDGE:\n{self.base_knowledge_to_string()}""",
        }

        instructions_message: ChatCompletionMessageParam | None = (
            {
                "role": "system",
                "content": f"""## INSTRUCTIONS:\n{self.instructions_to_string()}""",
            }
            if self.instructions
            else None
        )

        rules_message: ChatCompletionMessageParam = {
            "role": "system",
            "content": f"""## RULES:\n{self.rules_to_string()}""",
        }
        messages = [
            role_task_skill_message,
            base_knowledge_message,
            *conversation_messages,
            rules_message,
            instructions_message,
        ]

        return [message for message in messages if message is not None]


def get_openai_messages(question: str, answer: str, include_contact_info: bool = False, gender: str = "unknown"):
    faqs = search_faq(question)
    system_prompt_config = SystemPromptConfig()

    if include_contact_info:
        system_prompt_config.instructions.append(
            Instruction(
                content="You must ask the user for their contact information to provide better advice on the phone they are interested in.",
                examples=[
                    "Để tư vấn tốt hơn cho bạn, bạn có thể cho mình biết số điện thoại hoặc email của bạn không?",
                ],
            )
        )
    system_prompt_config.base_knowledge = [f"User gender: {gender}"] + system_prompt_config.base_knowledge
    if faqs:
        system_prompt_config.base_knowledge.append(
            f"Some frequently asked questions (FAQs) in the store:\n"
            + (
                "\n".join(
                    [
                        (
                            f"   - Question {i + 1}: {faq.question}\n"
                            f"Answer: {faq.answer}"
                        )
                        for i, faq in enumerate(faqs)
                    ]
                )
            )
        )
        system_prompt_config.rules.append(
            "If the latest user message is a question related to the FAQs, you should answer it based on the FAQs. Else, you should answer it based on the other BASE KNOWLEDGE and don't use the FAQs."
        )

    return [
        *system_prompt_config.get_openai_messages(
            conversation_messages=[
                {
                    "role": "user",
                    "content": question,
                },
            ]
        ),
        {
            "role": "assistant",
            "content": answer,
        },
    ]

In [28]:
# convert faqs_training_df into 5 df by faq_id
def write_to_jsonlines(
    data: dict | list[dict], file_path: str = "fine_tune_tone_dataset.jsonl"
):
    with jsonlines.open(file_path, "a") as writer:
        if type(data) is dict:
            writer.write(data)
        else:
            writer.write_all(data)


for faq_id in faq_ids:
    faq_df = faqs_training_df[faqs_training_df["faq_id"] == faq_id]
    faq_df = faq_df.drop(columns=["faq_id"])
    for row in list(faq_df.itertuples(index=False)):
        converted_question = getattr(row, 'converted_question', '')
        converted_answer = getattr(row, 'converted_answer', '')
        original_question = getattr(row, 'original_question', '')
        original_answer = getattr(row, 'original_answer', '')
        gender = getattr(row, 'user_gender', 'unknown')
        include_contact_info = getattr(row, 'include_ask_contact_info', False)
        write_to_jsonlines({"messages" : get_openai_messages(
            question=converted_question,
            answer=converted_answer,
            include_contact_info=include_contact_info,
            gender=gender
        )}, file_path=f"fine_tune_tone_faq_{faq_id}_dataset.jsonl")

2025-06-15 15:15:25 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-06-15 15:15:25 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-06-15 15:15:26 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-06-15 15:15:26 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-06-15 15:15:27 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-06-15 15:15:27 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-06-15 15:15:28 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-06-15 15:15:28 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-06-15 15:15:29 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-06-15 15:15:29 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-06-15 15:15:29 - HTTP Request: POST

In [ ]:
from openai import OpenAI

client = OpenAI()

job = client.fine_tuning.jobs.create(
    training_file="file-SnmiZz15RE9a3SUJZWE92r",
    model="ft:gpt-4o-mini-2024-07-18:personal:hello:Biepb0lp",
    method={
        "type": "dpo",
        "dpo": {
            "hyperparameters": {
                "beta": 0.1,
                "batch_size": 5,
                "learning_rate_multiplier": 0.5,
                "n_epochs": 3,
            },
        },
    },
)

2025-06-15 16:35:10 - HTTP Request: POST https://api.openai.com/v1/fine_tuning/jobs "HTTP/1.1 400 Bad Request"


BadRequestError: Error code: 400 - {'error': {'message': 'Invalid model gpt-4o-mini for method dpo', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_allowed'}}